Academic Integrity and Learning Statement

By submitting my work, I confirm that:

1. The code, analysis, and documentation in this notebook are my own work and reflect my own understanding.
2. I am prepared to explain all code and analysis included in this submission.

If I used assistance (e.g., AI tools, tutors, or other resources), I have:

- Clearly documented where and how external tools or resources were used in my solution.
- Included a copy of the interaction (e.g., AI conversation or tutoring notes) in an appendix.

I acknowledge that:

- I may be asked to explain any part of my code or analysis during evaluation.
- Misrepresenting assisted work as my own constitutes academic dishonesty and undermines my learning.

In [2]:
import numpy as np
import os
import pandas as pd
import multiprocessing
import subprocess

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import optuna
import mlflow
import tensorflow as tf
import torch
import platform

In [3]:
# Enable auto-reload extension
%load_ext autoreload
# Automatically reload all modules before executing code
%autoreload 2

In [4]:
import base_utils_logging
import proj_utils_data_loader
import proj_configs
import proj_utils
import proj_utils_feat_engg
import proj_utils_plots
import proj_utils_model

In [5]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

In [6]:
# Check software specs
dict_sw_version = {
    'python': os.popen('python --version').read().strip(),
    'numpy': np.__version__,
    'pandas': pd.__version__,
    'optuna': optuna.__version__,
    'mlflow': mlflow.__version__,
}

for key, value in dict_sw_version.items():
    print(f'{proj_utils_plots.beautify(key, 1)} version is: {proj_utils_plots.beautify(value)}')


python version is: Python 3.11.13
numpy version is: 1.26.4
pandas version is: 2.2.3
optuna version is: 4.4.0
mlflow version is: 2.18.0


In [7]:
# Check hardware specs
def get_mac_gpu_info():
    try:
        # Get system information about GPU
        result = subprocess.run(['system_profiler', 'SPDisplaysDataType'],
                              capture_output=True, text=True)
        return result.stdout
    except Exception as e:
        return f"Error getting GPU info: {e}"

# Check CPU cores
print(f'CPU cores available to use: {proj_utils_plots.beautify(str(multiprocessing.cpu_count()))}')

# Check MPS availability
print("TensorFlow GPU devices:", proj_utils_plots.beautify(tf.config.list_physical_devices('GPU')))
print(f"Processor: {proj_utils_plots.beautify(platform.processor())}")
print(f"Machine: {proj_utils_plots.beautify(platform.machine())}")

print("PyTorch MPS (Metal) Status:")
print(f"MPS available: {proj_utils_plots.beautify(torch.backends.mps.is_available())}")
print(f"MPS built: {proj_utils_plots.beautify(str(torch.backends.mps.is_built()))}")

# Get detailed GPU information
print("\nDetailed GPU Information:")
print(get_mac_gpu_info())

CPU cores available to use: 10
TensorFlow GPU devices: []
Processor: arm
Machine: arm64
PyTorch MPS (Metal) Status:
MPS available: True
MPS built: True

Detailed GPU Information:
Graphics/Displays:

    Apple M4:

      Chipset Model: Apple M4
      Type: GPU
      Bus: Built-In
      Total Number of Cores: 10
      Vendor: Apple (0x106b)
      Metal Support: Metal 3
      Displays:
        Color LCD:
          Display Type: Built-in Liquid Retina XDR Display
          Resolution: 3024 x 1964 Retina
          Main Display: Yes
          Mirror: Off
          Online: Yes
          Automatically Adjust Brightness: Yes
          Connection Type: Internal




In [9]:
base_utils_logging.setup_logging()

In [10]:
base_utils_logging.logger.info('Starting the application')

In [11]:
df_raw_train = proj_utils_data_loader.load_data(proj_configs.TRAIN_FILE)
df_raw_test = proj_utils_data_loader.load_data(proj_configs.TEST_FILE)
df_raw_train.shape, df_raw_test.shape

((1460, 81), (1459, 80))

In [12]:
df_raw_train.sample(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
85,86,60,RL,121.0,16059,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,260000
352,353,50,RL,60.0,9084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2008,ConLw,Normal,95000
495,496,30,C (all),60.0,7879,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdWo,NaN,0,11,2009,WD,Abnorml,34900


In [13]:
# TODO: uncomment and comment below insignificant col list
# insignificant_cols = ['Order', 'PID']
insignificant_cols = ['Id']
target_col = 'SalePrice'
ignorables_cols = insignificant_cols + [target_col]
ordinal_cols = ['LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']
temporal_cols_name_pattern = ['Yr', 'Year']

In [14]:
df_raw_all, df_raw_target = proj_utils_data_loader.merge_train_test_data(df_raw_train, df_raw_test, insignificant_cols, target_col)
df_raw_all.shape, df_raw_target.shape

((2919, 80), (1460,))

In [15]:
n_cat_cardinality_threshold = proj_configs.CATEGORICAL_CARDINALITY_THRESHOLD_ABS
threshold_type = 'ABS'
feature_categories = proj_utils_feat_engg.classify_columns(df=df_raw_train, n_cat_threshold=n_cat_cardinality_threshold, threshold_type=threshold_type, cols_to_ignore=ignorables_cols, temporal_cols_name_pattern=temporal_cols_name_pattern, ordinal_cols=ordinal_cols)

In [16]:
cols_num_continuous, n_num_continuous, cols_num_discrete, n_num_discrete, cols_cat_nominal, n_cat_nominal, cols_cat_ordinal, n_cat_ordinal, cols_object, n_object, cols_temporal, n_temporal, cols_binary, n_binary = proj_utils_feat_engg.get_cols_as_tuple(feature_categories)

n_total = df_raw_train.shape[1] - len(ignorables_cols)

print(f'='*80)
print(f'Total raw columns = {proj_utils_plots.beautify(len(df_raw_train.columns))} \nNumerical Continuous = {proj_utils_plots.beautify(n_num_continuous)} \nNumerical Discrete = {proj_utils_plots.beautify(n_num_discrete)} \nCategorical Nominal = {proj_utils_plots.beautify(n_cat_nominal)} \nCategorical Ordinal = {proj_utils_plots.beautify(n_cat_ordinal)} \nObject/String = {proj_utils_plots.beautify(n_object)} \nTemporal = {proj_utils_plots.beautify(n_temporal)} \nBinary = {proj_utils_plots.beautify(n_binary)}')

print(f'='*80)
print(f'Any inconsistencies detected?[True/False] = {proj_utils_plots.beautify("True", 3) if n_total != len(df_raw_train.columns) - len(ignorables_cols) else proj_utils_plots.beautify("False", 1)}')
print(f'='*80)

Total raw columns = 82 
Numerical Continuous = 17 
Numerical Discrete = 14 
Categorical Nominal = 17 
Categorical Ordinal = 18 
Object/String = 7 
Temporal = 4 
Binary = 3
Any inconsistencies detected?[True/False] = False


In [ ]:
# Calculate the number of NaN values for each column
nan_counts = df_raw.isna().sum()

# Filter only columns that have NaN values and sort by the number of NaNs
cols_with_nans = nan_counts[nan_counts > 0].index.tolist()
print(f'Columns with NaNs: = {proj_utils_plots.beautify(len(cols_with_nans))}/{proj_utils_plots.beautify(n_total)}')
print(f'And they are: {cols_with_nans}')

In [ ]:
df_cardinality = proj_utils_feat_engg.get_cardinality_df(df_raw)

In [ ]:
df_cardinality

In [ ]:
proj_utils_plots.plot_cardinality(df_cardinality, n_cat_cardinality_threshold, threshold_used=threshold_type, type_of_cols='all', figsize=(20, 6))

In [ ]:
# Zoom into cols having NaNs only
proj_utils_plots.plot_cardinality(df_cardinality[df_cardinality['col_name'].isin(cols_with_nans)], n_cat_cardinality_threshold, threshold_used=threshold_type, type_of_cols="NaN", figsize=(10, 6))

In [ ]:
df_raw[cols_num_continuous].isnull().sum().sort_values(ascending=False)

In [ ]:
# Creating a copy of the raw data to impute missing values for plotting purposes only (as NaNs are not plotted)
df_imputed_for_plots = df_raw.copy()
df_imputed_for_plots[cols_num_continuous] = df_imputed_for_plots[cols_num_continuous].fillna(0)
most_frequent = df_imputed_for_plots[cols_num_discrete].mode().iloc[0]
df_imputed_for_plots[cols_num_discrete] = df_imputed_for_plots[cols_num_discrete].fillna(most_frequent)

In [ ]:
df_imputed_for_plots

In [ ]:
df_imputed_for_plots_v2 = df_raw[cols_num_continuous+['SalePrice']].copy()

In [ ]:
df_imputed_for_plots_v2.sample(2)

In [ ]:
# correlation_plot = plot_correlation_with_demand(df, save_path="correlation_plot.png")
correlation_plot = proj_utils_plots.plot_correlation_with_target(df_imputed_for_plots_v2, target_col)

In [ ]:
proj_utils_plots.plot_numerical_distribution(df_imputed_for_plots, cols_num_continuous)

In [ ]:
proj_utils_plots.plot_categorical_distribution(df_imputed_for_plots, cols_cat_nominal)

In [ ]:
proj_utils_plots.plot_categorical_distribution(df_imputed_for_plots, cols_cat_ordinal)

In [ ]:
proj_utils_plots.plot_relationship_to_target(df_imputed_for_plots, cols_num_discrete, target_col)

In [ ]:
proj_utils_plots.plot_relationship_to_target(df_imputed_for_plots, cols_num_discrete, target_col, trend_type='median')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df_raw.drop(ignorables_cols, axis=1),
    df_raw[target_col],
    test_size=proj_configs.VALIDATION_SIZE,
    random_state=proj_configs.RANDOM_STATE
)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
num_columns = cols_num_continuous
cat_columns = cols_cat_nominal + cols_cat_ordinal + cols_num_discrete + cols_binary + cols_object
tempo_columns = cols_temporal

In [ ]:
len(num_columns), len(cat_columns), len(tempo_columns)

In [ ]:
pproc_pipe = proj_utils_feat_engg.create_pproc_pipeline(num_columns, cat_columns, tempo_columns)

In [ ]:
mlflow.login()

In [ ]:
# proj_utils_model.set_mlflow_uri('http://127.0.0.1:8080')
proj_utils_model.set_mlflow_uri("databricks")

In [ ]:
mlflow_experiment_name = f"/Users/asheesh.ambardar@live.com/{proj_configs.PROJECT_NAME}"
mlflow_experiment_id = proj_utils_model.get_or_create_experiment(mlflow_experiment_name)

In [ ]:
mlflow_experiment_id

In [ ]:
proj_utils_model.set_mlflow_experiment(mlflow_experiment_name)

In [ ]:
import warnings, logging
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings('ignore', category=ConvergenceWarning)
logging.getLogger("mlflow").setLevel(logging.ERROR)

In [ ]:
X_train_transformed = pproc_pipe.fit_transform(X_train)

In [ ]:
X_val_transformed = pproc_pipe.transform(X_val)

In [ ]:
y_train_transformed = y_train.to_numpy()
y_val_transformed = y_val.to_numpy()

In [ ]:
# Check for both NaN and None
has_nulls_or_nans = pd.isna(X_train_transformed).any()
print(f"Contains null or NaN values: {has_nulls_or_nans}")

In [ ]:
type(y_val)

In [ ]:
run_name='lasso-03'
artefact_path = 'artefact_path'
optimised_study_lasso = proj_utils_model.run_hyperparam_tuning_lasso(X_train, y_train, X_val, y_val, pproc_pipe, mlflow_experiment_id, run_name, artefact_path, proj_configs.OPTUNA_TRIAL_COUNT)

In [ ]:
run_name='xgb-02'
artefact_path = 'artefact_path'
optimised_study_xgb = proj_utils_model.run_hyperparam_tuning_xgb(X_train_transformed, y_train_transformed, X_val_transformed, y_val_transformed, mlflow_experiment_id, run_name, artefact_path, proj_configs.OPTUNA_TRIAL_COUNT)

In [ ]:
run_name='rfc-05'
artefact_path = 'artefact_path'
optimised_study_rfc = proj_utils_model.run_hyperparam_tuning_rfc(X_train, y_train, X_val, y_val, pproc_pipe, mlflow_experiment_id, run_name, artefact_path, proj_configs.OPTUNA_TRIAL_COUNT)

In [ ]:
model_uri = mlflow.get_artifact_uri(artefact_path)
model_uri

In [ ]:
loaded_model = mlflow.sklearn.load_model(
    model_uri="dbfs:/databricks/mlflow-tracking/1539464224853128/d72a53fe0a31467084ebeaceb1edc48e/artifacts/artefact_path"
)

In [ ]:
loaded_model

In [ ]:
df_raw_test = proj_utils_data_loader.load_data(proj_configs.TEST_FILE)

In [ ]:
df_test = proj_utils_data_loader.refactor_col_names(df_raw_test)

In [ ]:
df_test.sample(3)

In [ ]:
df_test.drop(insignificant_cols, axis=1, inplace=True)

In [ ]:
data_test_transformed = pproc_pipe.transform(df_test)
type(data_test_transformed)

In [ ]:
# Check for both NaN and None
has_nulls_or_nans = pd.isna(df_test_transformed).any()
print(f"Contains null or NaN values: {has_nulls_or_nans}")

In [ ]:
df_raw_train = df_raw.copy()
df_raw_train.drop(ignorables_cols, axis=1, inplace=True)

In [ ]:
data_train_transformed = pproc_pipe.transform(df_raw_train)
type(data_train_transformed)

In [ ]:
# Check for both NaN and None
has_nulls_or_nans = pd.isna(df_train_transformed).any()
print(f"Contains null or NaN values: {has_nulls_or_nans}")

In [ ]:
data_target = df_raw[target_col].to_numpy()
type(data_target)

In [ ]:
loaded_model.fit(data_train_transformed, data_target)

In [ ]:
train_preds = loaded_model.predict(data_train_transformed)
train_actuals = data_target

In [ ]:
# Evaluate the model
train_mse = mean_squared_error(train_actuals, train_preds).round(5)
train_r2 = r2_score(train_actuals, train_preds).round(5)

print("=== Model Performance ===")
print(f"Train MSE: {proj_utils_plots.beautify(train_mse)}, Train R2: {proj_utils_plots.beautify(train_r2)}")

In [ ]:
test_preds = loaded_model.predict(data_test_transformed)

In [ ]:
test_preds

In [ ]:
my_submission = pd.DataFrame({'Id': df_raw_test.Id, 'SalePrice': test_preds})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [ ]:
# this is a Trial object, not the underlying ML object.
best_performing_trial = optimised_study_xgb.best_trial
print(f'Best trial was at number {proj_utils_plots.beautify(str(best_performing_trial.number), 1)} with params as:\n {proj_utils_plots.beautify(str(best_performing_trial.params), 2)}')
print(f'Best score value is: {proj_utils_plots.beautify(str(best_performing_trial.value))}')

In [ ]:
best_performing_trial

In [ ]:
study_full_metrics = optimised_study_xgb.trials_dataframe()
# save the metrics to a file
proj_utils_model.save_hyperparams(f'full_metrics_{proj_utils.get_current_timestamp()}.csv', proj_configs.PATH_OUT_MODELS, study_full_metrics)

# peek at the full metrics dataframe
study_full_metrics

In [ ]:
# fetch number of trial runs per model type
num_lr_trials = study_full_metrics[study_full_metrics['params_model'] == 'lr'].shape[0]
num_lasso_trials = study_full_metrics[study_full_metrics['params_model'] == 'lasso'].shape[0]
num_ridge_trials = study_full_metrics[study_full_metrics['params_model'] == 'ridge'].shape[0]
num_elasticnet_trials = study_full_metrics[study_full_metrics['params_model'] == 'elasticnet'].shape[0]

print(f'Total trials = {proj_utils_plots.beautify(str(num_lr_trials + num_lasso_trials + num_ridge_trials + num_elasticnet_trials), 1)}\n-- LR trials = {proj_utils_plots.beautify(str(num_lr_trials), 1)}\n-- Lasso trials = {proj_utils_plots.beautify(str(num_lasso_trials), 1)}\n-- Ridge trials = {proj_utils_plots.beautify(str(num_ridge_trials), 1)}\n-- ElasticNet trials = {proj_utils_plots.beautify(str(num_elasticnet_trials), 1)}')

In [ ]:
# retrieve all performance values for each model type studied
grp_by_model_type_val = study_full_metrics.groupby('params_model')['value'].apply(list)
# retrieve the best performing model (use nsmallest if Optuna objective was to minimise,
grp_by_model_type_best_val = study_full_metrics.groupby('params_model')['value'].nsmallest(1)
# display the stats
grp_by_model_type_best_val

In [ ]:
 # grouping Optuna metrics by model type and using idxmax (or idmin) method to find a row with the best model performance (value) for each group
study_best_model_group = study_full_metrics.loc[study_full_metrics.groupby('params_model')['value'].idxmin()]

In [ ]:
study_best_model_group

In [ ]:
# retrieve the trial number of the best model for each model type - the Optuna metrics dataframe index and trial number are the same.
best_lr_trial = study_best_model_group[study_best_model_group['params_model'] == 'lr']['number'].values[0]
best_lasso_trial = study_best_model_group[study_best_model_group['params_model'] == 'lasso']['number'].values[0]
best_ridge_trial = study_best_model_group[study_best_model_group['params_model'] == 'ridge']['number'].values[0]
best_elasticnet_trial = study_best_model_group[study_best_model_group['params_model'] == 'elasticnet']['number'].values[0]

final_pipe_best_lr = models[best_lr_trial]
best_model_lr = final_pipe_best_lr.named_steps['regressor']
final_pipe_best_lasso = models[best_lasso_trial]
best_model_lasso = final_pipe_best_lasso.named_steps['regressor']
final_pipe_best_ridge = models[best_ridge_trial]
best_model_ridge = final_pipe_best_ridge.named_steps['regressor']
final_pipe_best_elasticnet = models[best_elasticnet_trial]
best_model_elasticnet = final_pipe_best_elasticnet.named_steps['regressor']

# retrieve the best model object (amongst all model types evaluated)
final_pipe_best = models[best_performing_trial.number]
best_model = final_pipe_best.named_steps['regressor']

In [ ]:
final_pipe_best

In [ ]:
final_pipe_best.fit(X_train, y_train)

In [ ]:
cols_final_inputs, cols_final_output_features = proj_utils_feat_engg.get_final_features(final_pipe_best, X_train)

In [ ]:
proj_utils_model.save_features(f'pproc_final_input_cols_{len(cols_final_inputs)}_{proj_utils.get_current_timestamp()}.csv', proj_configs.PATH_OUT_FEATURES, pd.DataFrame(cols_final_inputs))
proj_utils_model.save_features(f'pproc_final_output_features_{len(cols_final_output_features)}_{proj_utils.get_current_timestamp()}.csv', proj_configs.PATH_OUT_FEATURES, pd.DataFrame(cols_final_output_features))

In [ ]:
proj_utils_model.save_model(f'final_pipe_{proj_utils.get_current_timestamp()}.pkl', proj_configs.PATH_OUT_MODELS, final_pipe_best)

In [ ]:
y_train_preds = final_pipe_best.predict(X_train)
y_val_preds = final_pipe_best.predict(X_val)

In [ ]:
# Evaluate the model
train_mse = mean_squared_error(y_train, y_train_preds).round(5)
val_mse = mean_squared_error(y_val, y_val_preds).round(5)
train_r2 = r2_score(y_train, y_train_preds).round(5)
val_r2 = r2_score(y_val, y_val_preds).round(5)

print("=== Model Performance ===")
print(f"Train MSE: {proj_utils_plots.beautify(train_mse)}, Train R2: {proj_utils_plots.beautify(train_r2)}")
print(f"Validation MSE: {proj_utils_plots.beautify(val_mse)}, Validation R2: {proj_utils_plots.beautify(val_r2)}")

In [ ]:
string_to_log = f'=== Model Performance === \n Train MSE: {train_mse}, Train R2: {train_r2} \n Validation MSE: {val_mse}, Validation R2: {val_r2}'
proj_utils.save_file('metrics', 'validation_metrics.txt', proj_configs.PATH_OUT_MODELS, string_to_log)